In [ ]:
# importing required libraries

import pandas as pd
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from tabulate import tabulate

In [ ]:
# importing the dataset

file_path = '/content/drive/MyDrive/Cognifyz Internship/Dataset/Dataset.csv'
df_restaurants = pd.read_csv(file_path)

In [ ]:
# data preprocessing

df_restaurants = df_restaurants.drop(['Country Code', 'Locality', 'Locality Verbose',
                     'Currency', 'Has Table booking', 'Is delivering now',
                     'Switch to order menu', 'Price range', 'Rating color',
                     'Rating text', 'Average Cost for two', 'Has Online delivery'], axis = 1)
df_restaurants = df_restaurants.rename(columns = {'Restaurant ID': 'ID', 'Restaurant Name': 'name',
                                                  'City': 'city', 'Address': 'address', 'Longitude': 'longitude',
                                                  'Latitude': 'latitude', 'Cuisines': 'cuisines', 'Aggregate rating': 'rating',
                                                  'Votes': 'review_count'})
df_restaurants.head()

,ID,name,city,address,longitude,latitude,cuisines,rating,review_count
0,6317637,Le Petit Souffle,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...",121.027535,14.565443,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...",121.014101,14.553708,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...",121.056475,14.585318,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...",121.057508,14.584450,"Japanese, Korean",4.8,229


In [ ]:
# creating a function to top recommend restaurants in a specified location

def recommend_top_restaurants(data, user_location, radius_km=5, top_n=20):
    data['distance_km'] = data.apply(lambda row: geodesic((row['latitude'], row['longitude']), user_location).km, axis=1)
    # Filter restaurants within the specified radius
    nearby_restaurants = data[data['distance_km'] <= radius_km]
    # Sort restaurants by a combination of cuisine rating and number of reviews
    nearby_restaurants['combined_score'] = nearby_restaurants['rating'] * nearby_restaurants['review_count']
    nearby_restaurants = nearby_restaurants.sort_values(by='combined_score', ascending=False)
    # Select the top N restaurants
    top_restaurants = nearby_restaurants.head(top_n)
    if(top_restaurants.empty):
      print(f"No restaurants in your location in and around {radius_km} km.")
      return 0
    print(tabulate(top_restaurants[['name', 'city', 'distance_km', 'rating', 'review_count', 'cuisines', 'latitude', 'longitude']]))

In [ ]:
# creating a function which can get the coordinates of a city by passing its name to it

def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(location_name)
    if location:
        latitude, longitude = location.latitude, location.longitude
        return latitude, longitude
    else:
        return None

In [ ]:
location_name = input("Enter the city you're in: ")
user_location = get_coordinates(location_name)
# user_location = (11.9416,79.8083)
print(user_location)
isbool = recommend_top_restaurants(df_restaurants, user_location)
# its found that sometimes the coordinates from the function are not precise
# so in that case we cross check by giving the precise coordinates
if(isbool == None):
  coords = input("Give your coordinates for precise analysis: ")
  user_location = coords
  recommend_top_restaurants(df_restaurants, user_location)

Enter the city you're in: Pondicherry
(10.91564885, 79.80694879844232)
No restaurants in your location in and around 5 km.
None
Give your coordinates for precise analysis:11.9416,79.8083
----  ------------------------------  ----------  -------  ---  ---  ---------------------------------------------------------------  -------  -------
9141  Baker Street                    Puducherry  2.68962  4.1  791  Bakery, French                                                   11.9293  79.8296
9142  Zuka Choco-la                   Puducherry  2.71651  4.2  656  Desserts                                                         11.9298  79.8302
9146  Le Cafe                         Puducherry  3.18603  3.1  875  Cafe                                                             11.9315  79.8357
9145  Villa Shanti                    Puducherry  3.01411  3.9  535  Continental                                                      11.9297  79.8332
9152  The Indian Kaffe Express        Puducherry  3.14271 

<ipython-input-15-c5e5876ab53d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_restaurants['combined_score'] = nearby_restaurants['rating'] * nearby_restaurants['review_count']


In [ ]:
# creating a function to recommend top restaurants in a specified location and a specified cuisine

def recommend_top_restaurants_cuisines(data, user_location, cuisines_filter=None, radius_km=5, top_n=5):
    data['distance_km'] = data.apply(lambda row: geodesic((row['latitude'], row['longitude']), user_location).km, axis=1)
    nearby_restaurants = data[data['distance_km'] <= radius_km]
    if cuisines_filter:
        nearby_restaurants = nearby_restaurants[nearby_restaurants['cuisines'].str.contains(cuisines_filter, case=False)]

    # Sort restaurants by a combination of cuisines rating and number of reviews
    nearby_restaurants['combined_score'] = nearby_restaurants['rating'] * nearby_restaurants['review_count']
    nearby_restaurants = nearby_restaurants.sort_values(by='combined_score', ascending=False)
    top_restaurants = nearby_restaurants.head(top_n)
    if(top_restaurants.empty):
      print(f"No restaurants for {cuisines_filter} in your location in and around {radius_km} km.")
      return
    print(tabulate(top_restaurants[['name', 'city', 'distance_km', 'rating', 'review_count', 'cuisines', 'latitude', 'longitude']]))

In [ ]:
location_name = input("Enter the city you're in: ")
user_location = get_coordinates(location_name)
desired_cuisines = input("Which cuisine do you wanna try?: ")
# user_location = (11.9416,79.8083)
print(user_location)
isbool = recommend_top_restaurants(df_restaurants, user_location)
if(isbool == None):
  coords = input("Give your coordinates for precise analysis: ")
  user_location = coords
  recommend_top_restaurants(df_restaurants, user_location)

Enter the city you're in: Puducherry
Which cuisine do you wanna try?: Cafe
No restaurants for Cafe in your location in and around 5 km.
(10.91564885, 79.80694879844232)
No restaurants in your location in and around 5 km.
Give your coordinates for precise analysis: 11.9416,79.8083
----  ------------------------------  ----------  -------  ---  ---  ---------------------------------------------------------------  -------  -------
9141  Baker Street                    Puducherry  2.68962  4.1  791  Bakery, French                                                   11.9293  79.8296
9142  Zuka Choco-la                   Puducherry  2.71651  4.2  656  Desserts                                                         11.9298  79.8302
9146  Le Cafe                         Puducherry  3.18603  3.1  875  Cafe                                                             11.9315  79.8357
9145  Villa Shanti                    Puducherry  3.01411  3.9  535  Continental                                   

<ipython-input-15-c5e5876ab53d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_restaurants['combined_score'] = nearby_restaurants['rating'] * nearby_restaurants['review_count']
